# Patient / Doctor Scheduler

Create a patient class and a doctor class. Have a doctor that can handle multiple patients and setup a scheduling program where a doctor can only handle 16 patients during an 8 hr work day.

In [1]:
from datetime import datetime
from datetime import date
import time

In [2]:
# Create abstract Person class with name and appointment attributes and check calendar method
class Person:
    
    # Appointments attribute is a list that will store confirmed appointments
    def __init__(self,name):        
        self.name = name
        self.appointments = []
      
    # Check calendar method confirms if the requested time is available   
    def isavailable(self,time,calendar):
        
        # Search appointments list to confirm time is open 
        if time not in self.appointments:
            pass
        else:    
            print("Sorry, that time is not available. Please select another time.")

    # Add appointment method appends the booked appointment to patient's and doctors records
    def add_appointment(self,time,appointments = []):
        
        self.appointments.append(time)
        return self.appointments    
    
# Create Patient subclass that inherits from the Person class
class Patient(Person):
    
    def __init__(self,name):
        super().__init__(name)

# Create Doctor subclass inheriting from Person class 
class Doctor(Person):
    
    def __init__(self,name):
        super().__init__(name)

In [3]:
doctor = Doctor("Doctor X")

In [4]:
patient = Patient("Patient Y")

In [5]:

# Create Calendar to be used by doctor 
# Function generates 2021 calendar with combinations of months, days, hours and minutes
def create_calendar():
    
    # Create list to store calendar dates and times
    calendar = []

    # Store minutes in list
    minutes = [0,30]

    # Iterate through all 12 months
    for month in range(1,13):

        # Add 28 days to month of Feb except President's Day holiday (Feb 17)
        if month == 2:
            for day in range(1,29):
                for hour in range(8,16):
                    for i in range(len(minutes)):
                        timeslot = datetime(2021,month,day,hour,minutes[i]).strftime("%m/%d/%y %I:%M")
                        if day != 17:
                            calendar.append(timeslot)

        # Add 31 days to the months listed below
        # Remove holidays on Jan 1, May 31, July 4, Dec 25
        elif month in [1,3,5,7,8,10,12]:
            for day in range(1,32):
                for hour in range(8,16):
                    for i in range(len(minutes)):        
                        timeslot = datetime(2021,month,day,hour,minutes[i]).strftime("%m/%d/%y %I:%M")
                        holidays = ["01/01/21","05/31/21","07/04/21","12/25/21"]
                        if not any(x in timeslot for x in holidays):
                            calendar.append(timeslot)

        # Add 30 days to remaining months -- April, June, Sept, Nov
        elif month in [4,6,9,11]:
            for day in range(1,31):
                for hour in range(8,16):
                    for i in range(len(minutes)):        
                        timeslot = datetime(2021,month,day,hour,minutes[i]).strftime("%m/%d/%y %I:%M")
                        holidays = ["09/06/21","11/25/21"]
                        if not any(x in timeslot for x in holidays):
                            calendar.append(timeslot)

    # Return calendar so it can be passed on to functions below    
    return calendar


In [6]:
calendar = create_calendar()

In [7]:
len(calendar)

5728

In [8]:
# Create function to show available times on the select date in the calendar
# Calendar is passed into function as an argument
def show_times(patient,doctor,calendar):
    
    while True:

        # Ask patient/doctor to select an appointment date 
        # Append to doctor object for retrieval in functions below
        doctor.select_date = input("\nEnter the date when you would like to schedule an appointment using the mm/dd/yy format: ")

        # If selected date is available in calendar, display available times on that date
        if doctor.select_date in [timeslot[:8] for timeslot in calendar]:

            print(f"\nThe following times are available on {doctor.select_date}:")
            print(" ")

            # Display open time slots for the requested date
            for timeslot in calendar:

                # If date (first 7 characters) matches requested date, display the time (last 4 characters)
                if timeslot[:8] == doctor.select_date:
                    print(timeslot[-5:])
            
            # Break out of while loop after displaying times     
            break

        # Else display error message and return to top of look to ask user for requested date
        else:
            print("\nSorry, that date is invalid or unavailable.")
            continue
                  
    

In [9]:
apt_time = show_times(patient,doctor,calendar)


Enter the date when you would like to schedule an appointment using the mm/dd/yy format:  01/14/21



The following times are available on 01/14/21:
 
08:00
08:30
09:00
09:30
10:00
10:30
11:00
11:30
12:00
12:30
01:00
01:30
02:00
02:30
03:00
03:30


In [10]:
# Version w/o confirmation prompt

# Create function to book an appointment time
# Patient, doctor, calendar and selected date are passed into function as arguments
def schedule_appointment(patient,doctor,calendar):
    
    # Define scheduled variable that will be set to true once appointment has been scheduled
    
    while True:
        
        # Ask patient/doctor to select an appointment time
        # Selected time is stored with doctor object to enable retrieval in confirmation message
        doctor.select_time = input("\nEnter the time when you would like to schedule an appointment using the HH:MM format: ")
        
        # If selected time is available
        if doctor.select_time in [timeslot[-5:] for timeslot in calendar]:
        
            # Store selected date and time in tuple to support for loop below
            selections = (doctor.select_date,doctor.select_time)

            # Iterate through each index in calendar
            for i in range(len(calendar)):

                # If time slot contains selected date and time, add to doctor's and patient's appointment records
                # Uses add appointment method for doctor and patient objects to ensure records are kepts for each one
                # Then remove selected time slot from calendar
                if all(x in calendar[i] for x in selections):           

                    doctor.add_appointment(calendar[i])
                    patient.add_appointment(calendar[i])
                    calendar.pop(i)
                    return "Thank you, your appointment has been booked for {} at {}".format(doctor.select_date, doctor.select_time)
        
        # Else if unavailable time or invalid input, display error message and start over
        else:
            print("\nSorry, that time is unavailable or invalid.")
            continue      
            
  

In [11]:
# Version w/ confirmation prompt

# Create function to book an appointment time
# Patient, doctor, calendar and selected date are passed into function as arguments
def schedule_appointment(patient,doctor,calendar):
    
    # Define scheduled variable that will be set to true once appointment has been scheduled
    
    while True:
        
        # Ask patient/doctor to select an appointment time
        # Selected time is stored with doctor object to enable retrieval in confirmation message
        doctor.select_time = input("\nEnter the time when you would like to schedule an appointment using the HH:MM format: ")
        
        # If selected time is available
        if doctor.select_time in [timeslot[-5:] for timeslot in calendar]:
        
             # Ask user to confirm selection
            while True:
                
                doctor.confirm = input(f"You have selected {doctor.select_date} at {doctor.select_time}. Would you like to book this appointment? (Y or N)")
                
                # If confirmed, display confirmation
                if doctor.confirm[0].upper() == "Y":
                
                    # Store selected date and time in tuple to support for loop below
                    selections = (doctor.select_date,doctor.select_time)

                    # Iterate through each index in calendar
                    for i in range(len(calendar)):

                        # If time slot contains selected date and time, add to doctor's and patient's appointment records
                        # Uses add appointment method for doctor and patient objects to ensure records are kepts for each one
                        # Then remove selected time slot from calendar
                        if all(x in calendar[i] for x in selections):           

                            doctor.add_appointment(calendar[i])
                            patient.add_appointment(calendar[i])
                            calendar.pop(i)
                            return "Thank you, your appointment has been booked."
                        
                # If not, break on return to beginning
                elif doctor.confirm[0].upper() == "N":
                    return "Okay let's start over then."                  
                
                # Else
                else:
                    print("Sorry, I don't understand.")
                    continue
        
        # Else if unavailable time or invalid input, display error message and start over
        else:
            print("\nSorry, that time is unavailable or invalid.")
            continue      
            
  

In [12]:
doctor.select_date

'01/14/21'

In [14]:
schedule_appointment(patient,doctor,calendar)


Enter the time when you would like to schedule an appointment using the HH:MM format:  01:00
You have selected 01/14/21 at 01:00. Would you like to book this appointment? (Y or N) y


'Thank you, your appointment has been booked.'

In [15]:
doctor.appointments

['01/14/21 01:00']

In [16]:
patient.appointments

['01/14/21 01:00']

In [19]:
# Create modify function to allow patient/doctor to reschedule or cancel an appointment
# Patient, doctor, and calendar are passed into function as arguments
def modify_appointment(patient,doctor,calendar):
    
    # Define rescheduled, cancelled and exit variables that will be set to true based on outcome

    if len(patient.appointments) == 0:
        
        print("No appointments with doctor were found for this patient.")
                
    else:       
        # Show patient's confirmed appointmentin string format
        print("\nThe following appointment has been scheduled:")
        print(" ")
        print(", ".join(patient.appointments))

        while True:

            # Ask whether to cancel or reschedule 
            select = input("\nWould you like to reschedule or cancel this appointment? \nEnter 'exit' to keep your existing appointment.")

            # If cancel, remove appointment time from patient's and doctor's records and place it back in calendar
            # Set cancelled to True to display cancellation message to end program
            if "cancel" in select.lower():
                
                patient.appointments.pop()
                calendar.append(doctor.appointments.pop())
                
                return "Thank you, your appointment has been cancelled."             
                
            # If reschedule, remove appointment time from patient's and doctor's records and place it back in calendar
            # Set rescheduled to True to restart program and allow patient/doctor to schedule new appointment
            elif "reschedule" in select.lower():

                patient.appointments.pop()
                calendar.append(doctor.appointments.pop())
                return "Okay, let's reschedule your appointment."

            # If exit, set exit to True and break to terminate function without doing anything
            elif "exit" in select.lower():  
                return "Exiting program."
            
            # Else display error message and start over
            else:
                print("\nSorry, I didn't undestand.")
                continue

In [20]:
modify_appointment(patient,doctor,calendar)


The following appointment has been scheduled:
 
01/14/21 01:00



Would you like to reschedule or cancel this appointment? 
Enter 'exit' to keep your existing appointment. reschedule


"Okay, let's reschedule your appointment."

In [22]:
# Program structure

# Instantiate patient and doctor
doctor = Doctor("Dr Strangelove")
patient = Patient("Patient X")

# Create doctor's calendar
calendar = create_calendar()

print("Welcome to the appointment scheduling portal")

scheduling = True
    
while scheduling:
    
    option = input("\nPlease enter 'S' to schedule a new appointment or 'M' to modify an existing appointment. ")
    
    # If schedule:
    if option.upper()[0] == "S":
        
        # Call show times function to display available times for selected date
        # Show times returns the selected date
        apt_time = show_times(patient,doctor,calendar)
        
        # Call schedule appointment function to select a time and store output in variable to support "if-else" flow below
        # Function returns "scheduled" and adds appointment to doctor's and patient's records 
        book_apt = schedule_appointment(patient,doctor,calendar)
        
        # Display appointment confirmation (if applicable)
        print(book_apt)
                 
        
        # If scheduled, end program
        if "booked" in book_apt:
            scheduling = False            
            break
        
        # Else program will start over 
        elif "start over" in book_apt:
            option == "S"
            continue
          

    # If modify:
    elif option.upper()[0] == "M":    
    
        # Call modify function and store result in variable to support "if-else" flow below
        modify = modify_appointment(patient,doctor,calendar)
        
        if len(modify) > 0:
            print(modify)        
        
        # If there are no appointments, 
        if modify is None:
            scheduling = False
            break
        
        # If rescheduled, return to start of while loop and begin scheduling process
        elif "reschedule" in modify:
            option == "S"
            continue
            
        # Else if cancelled or exiting, end program        
        else:
            scheduling = False
            break
        
    # Else display error message and start over
    else:
        print("Sorry, I didn't understand.")
        continue
        


Welcome to the appointment scheduling portal



Please enter 'S' to schedule a new appointment or 'M' to modify an existing appointment.  m


You don't have any upcoming appointments with your doctor.
None
